In [9]:
import pandas as pd
import create_test_data
import boto3
from unsupervised_topic_segmentation import dataset, core


# AMI data cleaning

In [10]:
#words to remove
FILLERS=["um", "uh", "oh", "hmm", "mm-hmm", "uh-uh", "you know"]


In [11]:
#ami loading
topic_path='data/ami-and-icsi-corpora-master/ami-corpus/output/topics/'
topic_jsons=create_test_data.jsons_to_dict_list(topic_path)

#transcript ids
ami_ids=["AMI_"+str(x) for x in range(len(topic_jsons))]

#jsons to clean dfs
topic_dfs=[create_test_data.clean_topic_json(x,y,fillers=FILLERS) for x,y in zip(topic_jsons,ami_ids)]

#df with one sentence per row
df_ami=pd.concat(topic_dfs)

#df with one transcript per row
t_ami = pd.DataFrame({'transcript_id':ami_ids,
                      'sentences':[tuple(x['sentences']) for x in topic_dfs],
                      'length':[len(x['sentences']) for x in topic_dfs],
                      'topic_count':[tuple(x['topic_count']) for x in topic_dfs],
                      'mean_topic_length':[x.groupby(['topic_count']).size().mean() for x in topic_dfs],
                      'topic_desc':[tuple(x['topic_desc']) for x in topic_dfs],
                      'has_topic_desc':[tuple(x['has_topic_desc']) for x in topic_dfs]
                      })


In [4]:
#save to parquet
df_ami.to_parquet('data/ami.parquet')


# Transcript Data Loading

In [2]:
#transcript loading
t_cd = create_test_data.transcript_pickle_to_df()


In [5]:
#df with one sentence per row
transcript_ids=list(t_cd['transcript_id'])
sentences=list(t_cd['sentences'])
df_transcripts=pd.concat([pd.DataFrame({'transcript_id':x,'sentences':y}) for x,y in zip(transcript_ids,sentences)])

#clean transcripts
df_transcripts=dataset.preprocessing(df_transcripts,'sentences',FILLERS,min_caption_len=20)
df_transcripts=df_transcripts[["transcript_id","sentences"]]

In [23]:
#save to parquet
df_transcripts.to_parquet('data/transcripts.parquet')

# Load to S3 - requires credential access

In [ ]:
# Set the S3 bucket name and key for your CSV file
bucket_name = 'decoding-democracy-embed'
file_key = 'ami'

# Create an S3 client
s3 = boto3.client('s3')

# Upload the CSV file to S3
s3.upload_file('data/ami.parquet', bucket_name, 'ami.parquet')
s3.upload_file('data/transcript.parquet', bucket_name, 'transcript.parquet')


# Load Parquet

In [ ]:
filename = 'data/ami_embed_results/part-00000-aa8e8005-9cf8-48a2-a561-e03cf4754b85-c000.snappy.parquet'
df = create_test_data.snappy_parquet_to_df(filename)

In [ ]:
df['embeddings_len']=df['embeddings'].apply(lambda x: len(x))
df['length']=df['sentences'].apply(lambda x: len(x))
sum(df['length']!=df['embeddings_len'])
